# <font color= 'green'>   [01] Código para correlação das variáveis númericas </font>

In [ ]:
# Reset background style
sns.set_style('whitegrid')

# Calculate the correlation matrix excluding the 'CustomerID' column
corr = customer_data_cleaned.drop(columns=['CustomerID']).corr()

# Define a custom colormap
colors = ['#ff6200', '#ffcaa8', 'white', '#ffcaa8', '#ff6200']
my_cmap = LinearSegmentedColormap.from_list('custom_map', colors, N=256)

# Create a mask to only show the lower triangle of the matrix (since it's mirrored around its 
# top-left to bottom-right diagonal)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask, k=1)] = True

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr, mask=mask, cmap=my_cmap, annot=True, center=0, fmt='.2f', linewidths=2)
plt.title('Correlation Matrix', fontsize=14)
plt.show()

## Estas correlações elevadas indicam que estas variáveis ​​se movem muito próximas umas das outras, implicando um certo grau de multicolinearidade.

### principais impactos:

`Métrica de Distância:` Muitos algoritmos de clusterização, como K-means e hierárquico, dependem de métricas de distância (por exemplo, a distância euclidiana) para determinar a similaridade entre pontos de dados. A multicolinearidade pode distorcer essas distâncias, já que variáveis altamente correlacionadas podem dominar a métrica de distância, influenciando desproporcionalmente a formação dos clusters.


`Clusters Redundantes:` Variáveis altamente correlacionadas podem fornecer informações redundantes, resultando em clusters que refletem mais essa redundância do que diferenças reais entre os grupos de dados



# <font color='green'> [02] Pipeline </font>

- StandardScaler

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Suponha que `df_clean` seja seu DataFrame
numeric_features = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = df_clean.select_dtypes(include=['object']).columns.tolist()

# Defina os passos do pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputação para valores numéricos
    ('scaler', StandardScaler())  # Padronização para valores numéricos
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputação para valores categóricos usando a moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificação one-hot para variáveis categóricas
])

# Combinar etapas de pré-processamento para todos os tipos de features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),  # numeric_features são suas variáveis numéricas
        ('cat', categorical_transformer, categorical_features)  # categorical_features são suas variáveis categóricas
    ])

# Definir o pipeline completo
pipeline_stand = Pipeline(steps=[
    ('preprocessor', preprocessor),
])

# Agora você pode usar este pipeline para ajustar e transformar seus dados
pipeline_stand.fit(df_clean)
processed_data_stand = pipeline_stand.transform(df_clean)


- Normalizer

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Suponha que `df_clean` seja seu DataFrame
numeric_features = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = df_clean.select_dtypes(include=['object']).columns.tolist()

# Defina os passos do pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputação para valores numéricos
    ('normalizer', Normalizer())  # Normalização para valores numéricos
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputação para valores categóricos usando a moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificação one-hot para variáveis categóricas
])

# Combinar etapas de pré-processamento para todos os tipos de features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),  # numeric_features são suas variáveis numéricas
        ('cat', categorical_transformer, categorical_features)  # categorical_features são suas variáveis categóricas
    ])

# Definir o pipeline completo
pipeline_norm = Pipeline(steps=[
    ('preprocessor', preprocessor),
])

# Agora você pode usar este pipeline para ajustar e transformar seus dados
pipeline_norm.fit(df_clean)
processed_data_norm = pipeline_norm.transform(df_clean)


- Min-max Scaler

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Suponha que `df_clean` seja seu DataFrame
numeric_features = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = df_clean.select_dtypes(include=['object']).columns.tolist()

# Defina os passos do pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputação para valores numéricos
    ('normalizer', Normalizer())  # Normalização para valores numéricos
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputação para valores categóricos usando a moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificação one-hot para variáveis categóricas
])

# Combinar etapas de pré-processamento para todos os tipos de features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),  # numeric_features são suas variáveis numéricas
        ('cat', categorical_transformer, categorical_features)  # categorical_features são suas variáveis categóricas
    ])

# Definir o pipeline completo
pipeline_MinMax = Pipeline(steps=[
    ('preprocessor', preprocessor),
])

# Agora você pode usar este pipeline para ajustar e transformar seus dados
pipeline_MinMax.fit(df_clean)
processed_data_MinMax = pipeline_MinMax.transform(df_clean)


# <font color='green'> [03] PCA </font>

`Metodologia`

Aplicarei o PCA em todos os componentes disponíveis e traçarei a variação cumulativa explicada por eles. Este processo me permitirá visualizar quanta variação cada componente principal adicional pode explicar, ajudando-me assim a identificar o número ideal de componentes a serem retidos para a análise:

In [ ]:
# Setting CustomerID as the index column
customer_data_scaled.set_index('CustomerID', inplace=True)

# Apply PCA
pca = PCA().fit(customer_data_scaled)

# Calculate the Cumulative Sum of the Explained Variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

# Set the optimal k value (based on our analysis, we can choose 6)
optimal_k = 6

# Set seaborn plot style
sns.set(rc={'axes.facecolor': '#fcf0dc'}, style='darkgrid')

# Plot the cumulative explained variance against the number of components
plt.figure(figsize=(20, 10))

# Bar chart for the explained variance of each component
barplot = sns.barplot(x=list(range(1, len(cumulative_explained_variance) + 1)),
                      y=explained_variance_ratio,
                      color='#fcc36d',
                      alpha=0.8)

# Line plot for the cumulative explained variance
lineplot, = plt.plot(range(0, len(cumulative_explained_variance)), cumulative_explained_variance,
                     marker='o', linestyle='--', color='#ff6200', linewidth=2)

# Plot optimal k value line
optimal_k_line = plt.axvline(optimal_k - 1, color='red', linestyle='--', label=f'Optimal k value = {optimal_k}') 

# Set labels and title
plt.xlabel('Number of Components', fontsize=14)
plt.ylabel('Explained Variance', fontsize=14)
plt.title('Cumulative Variance vs. Number of Components', fontsize=18)

# Customize ticks and legend
plt.xticks(range(0, len(cumulative_explained_variance)))
plt.legend(handles=[barplot.patches[0], lineplot, optimal_k_line],
           labels=['Explained Variance of Each Component', 'Cumulative Explained Variance', f'Optimal k value = {optimal_k}'],
           loc=(0.62, 0.1),
           frameon=True,
           framealpha=1.0,  
           edgecolor='#ff6200')  

# Display the variance values for both graphs on the plots
x_offset = -0.3
y_offset = 0.01
for i, (ev_ratio, cum_ev_ratio) in enumerate(zip(explained_variance_ratio, cumulative_explained_variance)):
    plt.text(i, ev_ratio, f"{ev_ratio:.2f}", ha="center", va="bottom", fontsize=10)
    if i > 0:
        plt.text(i + x_offset, cum_ev_ratio + y_offset, f"{cum_ev_ratio:.2f}", ha="center", va="bottom", fontsize=10)

plt.grid(axis='both')   
plt.show()

Analisando a tabela de PCAs

In [ ]:
# Creating a PCA object with 6 components
pca = PCA(n_components=6)

# Fitting and transforming the original data to the new PCA dataframe
customer_data_pca = pca.fit_transform(customer_data_scaled)

# Creating a new dataframe from the PCA dataframe, with columns labeled PC1, PC2, etc.
customer_data_pca = pd.DataFrame(customer_data_pca, columns=['PC'+str(i+1) for i in range(pca.n_components_)])

# Adding the CustomerID index back to the new PCA dataframe
customer_data_pca.index = customer_data_scaled.index

Analisando as variaveis

In [ ]:
# Define a function to highlight the top 3 absolute values in each column of a dataframe
def highlight_top3(column):
    top3 = column.abs().nlargest(3).index
    return ['background-color:  #ffeacc' if i in top3 else '' for i in column.index]

# Create the PCA component DataFrame and apply the highlighting function
pc_df = pd.DataFrame(pca.components_.T, columns=['PC{}'.format(i+1) for i in range(pca.n_components_)],  
                     index=customer_data_scaled.columns)

pc_df.style.apply(highlight_top3, axis=0)

Visualizando os principais compopnentes :

In [ ]:
# Setting up the color scheme for the clusters (RGB order)
colors = ['#e8000b', '#1ac938', '#023eff']

In [ ]:
# Create separate data frames for each cluster
cluster_0 = customer_data_pca[customer_data_pca['cluster'] == 0]
cluster_1 = customer_data_pca[customer_data_pca['cluster'] == 1]
cluster_2 = customer_data_pca[customer_data_pca['cluster'] == 2]

# Create a 3D scatter plot
fig = go.Figure()

# Add data points for each cluster separately and specify the color
fig.add_trace(go.Scatter3d(x=cluster_0['PC1'], y=cluster_0['PC2'], z=cluster_0['PC3'], 
                           mode='markers', marker=dict(color=colors[0], size=5, opacity=0.4), name='Cluster 0'))
fig.add_trace(go.Scatter3d(x=cluster_1['PC1'], y=cluster_1['PC2'], z=cluster_1['PC3'], 
                           mode='markers', marker=dict(color=colors[1], size=5, opacity=0.4), name='Cluster 1'))
fig.add_trace(go.Scatter3d(x=cluster_2['PC1'], y=cluster_2['PC2'], z=cluster_2['PC3'], 
                           mode='markers', marker=dict(color=colors[2], size=5, opacity=0.4), name='Cluster 2'))

# Set the title and layout details
fig.update_layout(
    title=dict(text='3D Visualization of Customer Clusters in PCA Space', x=0.5),
    scene=dict(
        xaxis=dict(backgroundcolor="#fcf0dc", gridcolor='white', title='PC1'),
        yaxis=dict(backgroundcolor="#fcf0dc", gridcolor='white', title='PC2'),
        zaxis=dict(backgroundcolor="#fcf0dc", gridcolor='white', title='PC3'),
    ),
    width=900,
    height=800
)

# Show the plot
fig.show()

Distribuição dos Clusters

In [ ]:
# Calculate the percentage of customers in each cluster
cluster_percentage = (customer_data_pca['cluster'].value_counts(normalize=True) * 100).reset_index()
cluster_percentage.columns = ['Cluster', 'Percentage']
cluster_percentage.sort_values(by='Cluster', inplace=True)

# Create a horizontal bar plot
plt.figure(figsize=(10, 4))
sns.barplot(x='Percentage', y='Cluster', data=cluster_percentage, orient='h', palette=colors)

# Adding percentages on the bars
for index, value in enumerate(cluster_percentage['Percentage']):
    plt.text(value+0.5, index, f'{value:.2f}%')

plt.title('Distribution of Customers Across Clusters', fontsize=14)
plt.xticks(ticks=np.arange(0, 50, 5))
plt.xlabel('Percentage (%)')

# Show the plot
plt.show()

Outras Métricas de avaliação de Clusters :

| Métrica                          | Descrição                                                                                                                                                       |
|----------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Índice de Silhueta (Silhouette Score)   | O índice de silhueta mede a separação dos clusters. Um valor próximo de 1 indica clusters bem separados, enquanto valores próximos de 0 ou negativos indicam sobreposição entre clusters.                                                   |
| Calinski-Harabasz Score          | O índice Calinski-Harabasz é uma medida da dispersão entre os clusters em relação à dispersão dentro dos clusters. Valores mais altos indicam clusters mais bem definidos. Interpretação: Quanto maior o valor, melhor a definição dos clusters.                                     |
| Índice Davies-Bouldin (Davies Bouldin Score) | O índice Davies-Bouldin avalia a similaridade média entre cada cluster e seu cluster mais semelhante. Valores mais baixos indicam uma melhor separação entre os clusters. Interpretação: Quanto mais próximo de 0, melhor a separação entre os clusters.                                               |


| Métrica                                        | Fórmula                                                                                                                                                       |
|------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Índice de Silhueta (Silhouette Score)          | \( s(i) = \frac{b(i) - a(i)}{\max\{a(i), b(i)\}} \), onde \( a(i) \) é a distância média do ponto \( i \) para todos os outros pontos no mesmo cluster e \( b(i) \) é a menor distância média do ponto \( i \) para todos os pontos em um cluster diferente. |
| Calinski-Harabasz Score                        | \( CH = \frac{SS_{\text{between}}}{SS_{\text{within}}} \times \frac{N - K}{K - 1} \), onde \( SS_{\text{between}} \) é a soma dos quadrados das distâncias entre a média de cada cluster e a média global dos dados, e \( SS_{\text{within}} \) é a soma dos quadrados das distâncias de cada ponto ao centroide do seu próprio cluster. |
| Índice Davies-Bouldin (Davies Bouldin Score)   | \( DB = \frac{1}{K} \sum_{i=1}^{K} \max_{i \neq j} \left( \frac{a(i) + a(j)}{R_{ij}} \right) \), onde \( K \) é o número de clusters, \( a(i) \) é a similaridade intra-cluster do cluster \( i \), definida como a média das distâncias de todos os pontos do cluster ao centroide do cluster \( i \), e \( R_{ij} \) é a similaridade entre os clusters \( i \) e \( j \).           |


In [ ]:
# Compute number of customers
num_observations = len(customer_data_pca)

# Separate the features and the cluster labels
X = customer_data_pca.drop('cluster', axis=1)
clusters = customer_data_pca['cluster']

# Compute the metrics
sil_score = silhouette_score(X, clusters)
calinski_score = calinski_harabasz_score(X, clusters)
davies_score = davies_bouldin_score(X, clusters)

# Create a table to display the metrics and the number of observations
table_data = [
    ["Number of Observations", num_observations],
    ["Silhouette Score", sil_score],
    ["Calinski Harabasz Score", calinski_score],
    ["Davies Bouldin Score", davies_score]
]

# Print the table
print(tabulate(table_data, headers=["Metric", "Value"], tablefmt='pretty'))

# RADAR

In [ ]:
colors = ['#e8000b', '#1ac938', '#023eff']

In [ ]:
# Setting 'CustomerID' column as index and assigning it to a new dataframe
# dfdf = customer_data_cleaned.set_index('CustomerID')

# Standardize the data (excluding the cluster_label column)
scaler = StandardScaler()
dfdf_standardized = scaler.fit_transform(dfdf.drop(columns=['cluster_label'], axis=1))

# Create a new dataframe with standardized values and add the cluster_label column back
dfdf_standardized = pd.DataFrame(dfdf_standardized, columns=dfdf.columns[:-1], index=dfdf.index)
dfdf_standardized['cluster_label'] = dfdf['cluster_label']

# Calculate the centroids of each cluster_label
cluster_label_centroids = dfdf_standardized.groupby('cluster_label').mean()

# Function to create a radar chart
def create_radar_chart(ax, angles, data, color, cluster_label):
    # Plot the data and fill the area
    ax.fill(angles, data, color=color, alpha=0.4)
    ax.plot(angles, data, color=color, linewidth=2, linestyle='solid')
    
    # Add a title
    ax.set_title(f'cluster_label {cluster_label}', size=20, color=color, y=1.1)

# Set data
labels=np.array(cluster_label_centroids.columns)
num_vars = len(labels)

# Compute angle of each axis
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()

# The plot is circular, so we need to "complete the loop" and append the start to the end
labels = np.concatenate((labels, [labels[0]]))
angles += angles[:1]

# Initialize the figure
fig, ax = plt.subplots(figsize=(20, 10), subplot_kw=dict(polar=True), nrows=1, ncols=3)

# Create radar chart for each cluster_label
for i, color in enumerate(colors):
    data = cluster_label_centroids.loc[i].tolist()
    data += data[:1]  # Complete the loop
    create_radar_chart(ax[i], angles, data, color, i)

# Add input data
ax[0].set_xticks(angles[:-1])
ax[0].set_xticklabels(labels[:-1])

ax[1].set_xticks(angles[:-1])
ax[1].set_xticklabels(labels[:-1])

ax[2].set_xticks(angles[:-1])
ax[2].set_xticklabels(labels[:-1])

# Add a grid
ax[0].grid(color='grey', linewidth=0.5)

# Display the plot
plt.tight_layout()
plt.show()